Copyright The Numerical Algorithms Group Limited 1991.
 This file contains some functions that compute LUP factorizations of
 matrices over a field.  The main function to call is lupFactor.  It
 accepts one argument, which should be a non-singular square matrix.
 If the matrix is not square, "failed" will be returned.  If the matrix
 is non-singular, a 'cannot coerce "failed"' error will be displayed.
 lupFactor returns a Union(List Matrix field,"failed") object.  Coerce
 this to a List Matrix field before you try to use it.  See the comment
 before the definition of lupFactor for the reference for the
 algorithm.


 state the field here


In [0]:
)clear all

   All user variables and function definitions have been cleared.


 next computes a permutation matrix for mult on the right


In [1]:
field := Fraction Integer

   Fraction(Integer)
                                                                   Type: Type


In [2]:
permMat: (INT, INT, INT) -> Matrix field

                                                                   Type: Void


 find first col in first row that is nonzero or returns 0


In [3]:
permMat(dim, i, j) ==
  m : Matrix field :=
    diagonalMatrix [(if i = k or j = k then 0 else 1) for k in 1..dim]
  m(i,j) := 1
  m(j,i) := 1
  m

                                                                   Type: Void


In [4]:
nonZeroCol: Matrix field -> INT

                                                                   Type: Void


 this embeds the given square matrix in a larger square matrix
 where the extra space is filled with 1s on the diagonal, 0 elsewhere.


In [5]:
nonZeroCol(m) ==
  foundit := false
  col := 1
  for i in 1..ncols(m) while not foundit repeat
    for j in 1..nrows(m) while not foundit repeat
      if not(m(j,i) = 0) then
        col := i
        foundit := true
  col

                                                                   Type: Void


In [6]:
embedMatrix: (Matrix field,NNI,NNI) -> Matrix field

                                                                   Type: Void


 following implements algorithm in "The Design and Analysis of
 Computer Algorithms" by Aho, Hopcroft and Ullman


In [7]:
embedMatrix(m, oldDim, newDim) ==
  n := diagonalMatrix([1 for i in 1..newDim])$(Matrix(field))
  setsubMatrix!(n,1,1,m)
  n

                                                                   Type: Void


In [8]:
lupFactorEngine: (Matrix field, INT, INT)  -> List Matrix field

                                                                   Type: Void


 next computes floor of log base 2 of an integer


In [9]:
lupFactorEngine(a, m, p) ==
  m = 1 =>
    l : Matrix field := diagonalMatrix [1]
    pm : Matrix field := permMat(p,1,nonZeroCol a)
    [l,a*pm,pm]
  m2 : NNI := m quo 2
  b : Matrix field := subMatrix(a,1,m2,1,p)
  c : Matrix field := subMatrix(a,m2+1,m,1,p)
  lup := lupFactorEngine(b,m2,p)
  l1 := lup.1
  u1 := lup.2
  pm1 := lup.3
  d : Matrix field := c * (inverse(pm1) :: Matrix(field))
  e : Matrix field := subMatrix(u1,1,m2,1,m2)
  f : Matrix field := subMatrix(d,1,m2,1,m2)
  g : Matrix field := d - f * (inverse(e) :: Matrix(field)) * u1
  pmin2 : NNI := p - m2
  g' : Matrix field := subMatrix(g,1,nrows(g),p - pmin2 + 1,p)
  lup := lupFactorEngine(g',m2,pmin2)
  l2 := lup.1
  u2 := lup.2
  pm2 := lup.3
  pm3 := horizConcat(zero(pmin2,m2)$(Matrix field), pm2)
  pm3 := vertConcat(horizConcat(diagonalMatrix [1 for i in 1..m2],
    zero(m2,pmin2)$(Matrix field)),pm3)
  h : Matrix field := u1 * (inverse(pm3) :: Matrix(field))
  l : Matrix field := horizConcat(l1, zero(m2,m2)$(Matrix field))
  l := vertConcat(l,horizConcat(f * (inverse(e) :: Matrix(field)), l2))
  u : Matrix field := horizConcat(zero(m2,m2)$(Matrix field), u2)
  u := vertConcat(h,u)
  pm := pm3 * pm1
  [l,u,pm]

                                                                   Type: Void


In [10]:
intLog2: NNI -> NNI

                                                                   Type: Void


 here is the function to call


In [11]:
intLog2 n == if n = 1 then 0 else 1 + intLog2(n quo 2)

                                                                   Type: Void


In [12]:
lupFactor: Matrix field -> Union(List Matrix field,"failed")

                                                                   Type: Void


 Example from Aho, et al.


In [13]:
lupFactor m ==
  not((r := nrows m) = ncols m) =>
    messagePrint("Matrix must be square")$OUTFORM
    "failed"
  ilog := intLog2(2)
  not(r = 2 ^ ilog) =>
    m := embedMatrix(m,r,(n := 2 ^ (ilog + 1)))
    l := lupFactorEngine(m,n,n)
    [subMatrix(l.1,1,r,1,r),subMatrix(l.2,1,r,1,r),
      subMatrix(l.3,1,r,1,r)]
  lupFactorEngine(m,r,r)

                                                                   Type: Void


In [14]:
m : Matrix field := zero(4,4)

   +0  0  0  0+
   |          |
   |0  0  0  0|
   |          |
   |0  0  0  0|
   |          |
   +0  0  0  0+
                                              Type: Matrix(Fraction(Integer))


In [15]:
for i in 4..1 by -1 repeat m(5-i,i) := i

                                                                   Type: Void


In [16]:
m

   +0  0  0  4+
   |          |
   |0  0  3  0|
   |          |
   |0  2  0  0|
   |          |
   +1  0  0  0+
                                              Type: Matrix(Fraction(Integer))


 Example where the dimension does not start out a power of 2


In [17]:
lupFactor m

   Compiling function intLog2 with type NonNegativeInteger -> 
      NonNegativeInteger 
   Compiling function embedMatrix with type (Matrix(Fraction(Integer)),
      NonNegativeInteger,NonNegativeInteger) -> Matrix(Fraction(Integer
      )) 
   Compiling function nonZeroCol with type Matrix(Fraction(Integer))
       -> Integer 
   Compiling function permMat with type (Integer,Integer,Integer) -> 
      Matrix(Fraction(Integer)) 
   Compiling function lupFactorEngine with type (Matrix(Fraction(
      Integer)),Integer,Integer) -> List(Matrix(Fraction(Integer))) 
   Compiling function lupFactor with type Matrix(Fraction(Integer)) -> 
      Union(List(Matrix(Fraction(Integer))),"failed") 
   Compiling function G11373 with type Integer -> Boolean 


    +1  0  0  0+ +4  0  0  0+ +0  0  0  1+
    |          | |          | |          |
    |0  1  0  0| |0  3  0  0| |0  0  1  0|
   [|          |,|          |,|          |]
    |0  0  1  0| |0  0  2  0| |0  1  0  0|
    |          | |          | |          |
    +0  0  0  1+ +0  0  0  1+ +1  0  0  0+
                             Type: Union(List(Matrix(Fraction(Integer))),...)


In [18]:
m := [[1,2,3],[2,3,1],[3,1,2]]

   +1  2  3+
   |       |
   |2  3  1|
   |       |
   +3  1  2+
                                              Type: Matrix(Fraction(Integer))


INDEX-TOO-LARGE-ERROR: 
  #<SB-KERNEL:INDEX-TOO-LARGE-ERROR expected-type: (INTEGER 0 (0)) datum: 0>
